# Yield's Question Answering (QA) bot

### High-level approach
#### Pre-processing
- Load files from the repo, chunk them based on langugage-specific separators to maintain enough context
- Categorize each chunk as 'general' or 'technical' based on content type to allow semantic search to filter on query's underlying intent (general usage vs coding/technical)
- Add chunks to vector DB


#### Query time
- Call LLM to categorize the query as 'general' or 'technical' based on intent
- Run semantic search with category filter on vector db to get relevant chunks for LLM context
- Call LLM to answer user query with context


### Improvements
#### Parameters to explore to improve quality
- chunk size
- top-k count
- LLMs with larger context window (GPT-4 vs GPT-3.5 16k)
- Alternative Code-optimized LLMs such as BigCode https://huggingface.co/bigcode

#### Code suggestions
Code suggesstions could be improved by indexing [COOKBOOK](https://github.com/yieldprotocol/addendum-docs/blob/main/COOKBOOK.md) as the V2 documentation does not have extensive code examples

In [1]:
# Install dependencies
!pip install langchain openai chromadb GitPython ipython tiktoken

In [2]:
import os
from langchain.document_loaders import GitLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain, RetrievalQA
from IPython.display import display
from IPython.display import Markdown
from getpass import getpass
from pathlib import Path
from langchain.callbacks import StdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

stdout_handler = StdOutCallbackHandler() 

In [3]:
OPENAI_API_KEY = getpass()

········


In [4]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

## Common Functions

### Load contents from repo

In [5]:
def load_repo(remote_repo_url, local_repo_path, branch, file_filter=None):
    local_repo_exists = Path(local_repo_path).is_dir()

    if local_repo_exists:
        loader = GitLoader(
            repo_path=local_repo_path,
            branch=branch,
            file_filter=file_filter
        ) 
    else:
        loader = GitLoader(
            clone_url=remote_repo_url,
            repo_path=local_repo_path,
            branch=branch,
            file_filter=file_filter
        )
    return loader.load()

### Split document into chunks based on  Programming Language separators/syntax
* Split on programming language separators/syntax
* Categorize each chunk as 'general' or 'technical' based on content type

In [6]:
def split_docs(docs, language, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter.from_language(language=language, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    all_splits=[]
    all_metadatas=[]
    for d in docs:
        doc_file=d.page_content   
        metadata = d.metadata
        splits = text_splitter.split_text(doc_file)
        
        if 'cookbook' in metadata['file_path'].lower():
            metadata['category'] = "technical"
        else:
            metadata['category'] = "general"
        
        metadatas = [metadata for _ in splits]
        all_splits += splits
        all_metadatas += metadatas
        
    return {
        'all_splits': all_splits,
        'all_metadatas': all_metadatas
    }
    

## QA Workflow


In [7]:
# documentation repo
remote_repo_docsV2_url="https://github.com/yieldprotocol/docs-v2"
local_repo_docsV2_path="/tmp/yield_docs_v2_repo"

# cookbook repo
remote_repo_addendum_url="https://github.com/yieldprotocol/addendum-docs"
local_repo_addendum_path="/tmp/yield_addendum-docs"


branch="main"
file_filter=lambda file_path: file_path.endswith(".md")

In [8]:
chunk_size_chars = 2000
chunk_overlap_chars = 0

documentation_docs = load_repo(remote_repo_docsV2_url, local_repo_docsV2_path, branch, file_filter)
addendum_docs = load_repo(remote_repo_addendum_url, local_repo_addendum_path, branch, file_filter)

In [9]:
documentation_splits = split_docs(documentation_docs, Language.MARKDOWN, chunk_size_chars, chunk_overlap_chars)
addendum_splits = split_docs(addendum_docs, Language.MARKDOWN, chunk_size_chars, chunk_overlap_chars)

In [10]:
vector_db = Chroma(embedding_function=OpenAIEmbeddings())
documentation_splits_ids = vector_db.add_texts(documentation_splits['all_splits'], documentation_splits['all_metadatas'])
addendum_splits_ids = vector_db.add_texts(addendum_splits['all_splits'], addendum_splits['all_metadatas'])

In [11]:
categorization_prompt_template = """
You are a Web3 expert who is able to answer any user query on Yield protocol's documentation, code, whitepapers and many other such topics.

# INSTRUCTIONS
- Classify the user's query into one of these categories - 'general' or 'technical' or 'na'
- Use the examples below as reference, do not make up any categories.
- Always return the category in plain text format.
- If you are unable to process the query, just return 'na'

QUERY: How do I borrow
ANSWER: general

QUERY: How do I borrow using code
ANSWER: technical

QUERY: How do I integrate
ANSWER: technical

QUERY: How is lending rate calculated
ANSWER: general

QUERY: {query}
ANSWER:"""

QUERY_CATEGORIZATION_PROMPT = PromptTemplate(
    template=categorization_prompt_template, input_variables=["query"]
)

In [12]:
final_answer_prompt_template = """
You are a Web3 expert who is able to answer any user query on Yield protocol's documentation, code, whitepapers and many other such topics.

# INSTRUCTIONS
- The user's query will be wrapped in triple back ticks
- Only answer the query using the provided context below which may include general information and code, do not make up any information.
- If the query is related to code or integration, provide a step by step explanation on the process, use code suggestions whereever relevant and always use markdown format annotated with the language to show the code.
- For code suggestions, use comments to explain each important concept, class, variable, function and parameter.
- Yield Protocol has no JS SDK so always use ethers package for JS code suggestions.

# CONTEXT
{context}

# QUERY
```
{question}
```
"""
FINAL_ANSWER_PROMPT = PromptTemplate(
    template=final_answer_prompt_template, input_variables=["context", "question"]
)

In [13]:
def ask(query):
    display(Markdown(f"### Query\n{query}"))
    
    # 1. Call LLM to categorize query based on intent
    categorization_llm_chain = LLMChain(
        llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
        prompt=QUERY_CATEGORIZATION_PROMPT
    )

    with get_openai_callback() as cb:
        result = categorization_llm_chain.run(query=query)
        category = result.replace("'", "")
        display(Markdown(f"### Query Category\n**{category}**"))
        print(cb)

    # 2. Call LLM to answer user's query
    chain_type_kwargs = {"prompt": FINAL_ANSWER_PROMPT}
    qa_chain = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(temperature=0, model="gpt-4"), 
        chain_type="stuff", 
        retriever=vector_db.as_retriever(search_kwargs = {
            'k': 5,
            'filter': {'category': category}

        }), 
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )

    with get_openai_callback() as cb:
        answer = qa_chain({'query': query})
        display(Markdown("### Final Answer"))
        display(Markdown(answer['result']))
        print(cb)
        display(Markdown("### Sources"))
        for i, d in enumerate(answer['source_documents']):
            display(Markdown(f"**[Source {i+1}]**"))
            display(Markdown(d.page_content))
            display(Markdown(f"*File path: {d.metadata['file_path']}*"))


In [14]:
ask("how do i borrow using code")

### Query
how do i borrow using code

### Query Category
**technical**

Tokens Used: 171
	Prompt Tokens: 170
	Completion Tokens: 1
Successful Requests: 1
Total Cost (USD): $0.000257


### Final Answer

To borrow using code, you can follow the example provided in the Yield Protocol documentation for borrowing underlying. Here's a step-by-step guide on how to do it:

1. Import the necessary packages and set up your Ethereum provider and signer.

```javascript
const { ethers } = require("ethers");

// Set up your Ethereum provider and signer
const provider = new ethers.providers.JsonRpcProvider("YOUR_RPC_URL");
const signer = new ethers.Wallet("YOUR_PRIVATE_KEY", provider);
```

2. Set the contract addresses and ABI for the Ladle contract.

```javascript
const ladleAddress = "LADLE_CONTRACT_ADDRESS";
const ladleABI = "LADLE_CONTRACT_ABI";
```

3. Create an instance of the Ladle contract.

```javascript
const ladle = new ethers.Contract(ladleAddress, ladleABI, signer);
```

4. Define the parameters for the borrow action.

```javascript
const vaultId = "YOUR_VAULT_ID";
const receiver = "RECEIVER_ADDRESS";
const borrowed = ethers.utils.parseUnits("BORROWED_AMOUNT", "TOKEN_DECIMALS");
const maximumDebt = ethers.utils.parseUnits("MAXIMUM_DEBT_AMOUNT", "TOKEN_DECIMALS");
```

5. Execute the borrow action using the `ladle.batch()` function.

```javascript
async function borrow() {
  const tx = await ladle.batch([
    ladle.serveAction(vaultId, receiver, 0, borrowed, maximumDebt),
  ]);
  await tx.wait();
}

borrow().then(() => console.log("Borrowed successfully!"));
```

Replace the placeholders with the appropriate values for your use case. This code snippet will execute the borrow action, borrowing the specified amount of fyToken from the vault and sending it to the receiver address. The maximum debt parameter ensures that the debt incurred in the vault is within the provided limits.

...

### Borrow underlying

This action borrows fyToken from an existing vault, which is then exchanged for underlying in a YieldSpace pool. The amount of underlying obtained is an exact number provided as a parameter, and the debt incurred in the vault is variable but within provided limits. It can be combined with previous actions that create vaults and post collateral, among others.

```
  await ladle.batch([
    ladle.serveAction(vaultId, receiver, 0, borrowed, maximumDebt),
  ])
```

|Param  | Description|
|--------------|------------------------------------------------------------------------------------|
| `  vaultId  `   | Vault to add the collateral to. Set to 0 if the vault was created as part of this same batch.      |
| `  receiver  `   | Receiver of the collateral.      |
| `  0  `   | Collateral change, zero in this case      |
| `  borrowed  `   | Amount of debt to add to the vault, and fyTokens to send to the receiver.      |
| `  ladle  `   | Maximum debt to accept for the vault in fyToken terms.      |

*File path: COOKBOOK.md*

**[Source 5]**

```
                                        __________________   __________________
                                    .-/|                  \ /                  |\-.
                                    ||||                   |                   ||||
                                    ||||                   |                   ||||
                                    ||||                   |                   ||||
                                    ||||      Yield        |   "Recipes        ||||
                                    ||||                   |   made with love  ||||
                                    ||||     COOKBOOK      |   just like mama  ||||
                                    ||||                   |   used to make"   ||||
                                    ||||                   |                   ||||
                                    ||||                   |                   ||||
                                    ||||                   |                   ||||
                                    ||||__________________ | __________________||||
                                    ||/===================\|/===================\||
                                    `--------------------~___~-------------------''
```

*File path: COOKBOOK.md*

In [16]:
ask("How do i borrow")

### Query
How do i borrow

### Query Category
**general**

Tokens Used: 169
	Prompt Tokens: 168
	Completion Tokens: 1
Successful Requests: 1
Total Cost (USD): $0.000254


### Final Answer

To borrow on Yield Protocol, follow these steps:

1. Access the Yield v2 App at [https://app.yieldprotocol.com/#/borow](https://app.yieldprotocol.com/#/borow).
2. Connect your Ethereum wallet (e.g., Metamask) and ensure you have some ETH to pay for transaction fees and the base asset you'd like to use as collateral.
3. Choose an asset you want to borrow.
4. Add collateral to your vault.
5. Review and initiate the transaction.

Once the transaction is confirmed, you'll have successfully borrowed the asset. Remember that all loans in Yield require overcollateralization, so maintain a sufficient amount of collateral in your vault at all times to avoid liquidation.

...

# Lending

To lend, you first access the "Lend" tab in the [Yield v2 App](https://app.yieldprotocol.com/#/lend). Read our [Quick Start Guide](https://medium.com/yield-protocol/yield-protocol-v2-quickstart-guide-e516a955a405) for instructions. 

When you lend in Yield, you are buying future cash payments at a discount. These future cash payments are represented by tokens that we call “fyTokens”. A fyToken is a token that can be redeemed one-for-one for a base asset on some future date. FyTokens don’t pay interest themselves, rather the interest is determined by the difference between the face value of the token and the price you pay for it. 

The fixed interest rate you receive when lending is determined by a built-in automated market, and the more you lend, the lower your interest rate may be. 


## Lending Under the Hood

To lend at a fixed rate, you simply buy fyTokens at a discount to their face value. The discount you receive is equivalent to locking in a fixed return that can be calculated based on the time until a fyToken can be redeemed. 

For example, if on September 31, 2021, you buy 100 fyDai that matures in December 2021 for 98.8 Dai you will earn an implied rate of interest of 5% APR.

fyTokens can be held until the maturity date, upon which they may be redeemed for principal plus interest.

You can also exit your lending position early by selling your fyToken for an underlying asset. Because fyTokens are traded freely, changes in interest rates may affect the amount of underlying assets you receive when redeeming early.

To compensate lenders who do not redeem fyTokens right away, after maturity they begin earning interest in the form of an increasing redemption rate.

 <!-- TODO: need to expand more as how interest paid will be calculated after maturity. -->

*File path: users/lending.md*

**[Source 5]**

### A note on collateral

Borrowers must maintain a minimum amount of collateral in the system to secure the debt they owe. 

If a borrower fails to do so, they may be liquidated: their collateral will be seized and auctioned off to repay their debts. 

<!-- For ETH collateral, fyDAI uses the same collateralization ratio as MakerDAO, which is currently 150%. -->

<!-- TODO: Need to expand upon what ratio will be used for different collateral and different assets -->

[Edit this page](https://github.com/yieldprotocol/docs-v2/edit/main/users/borrowing.md)

*File path: users/borrowing.md*